# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 8 2023 10:30AM,256914,19,2022102835,"GUSA Granules USA, Inc.",Released
1,Feb 8 2023 10:26AM,256913,21,FF Preparation,"NBTY Global, Inc.",Released
2,Feb 8 2023 10:24AM,256912,10,0086363879,ISDIN Corporation,Released
3,Feb 8 2023 10:24AM,256912,10,0086363894,ISDIN Corporation,Released
4,Feb 8 2023 10:24AM,256911,16,SHL-9132360,"SHL Pharma, LLC",Released
5,Feb 8 2023 10:15AM,256910,10,SNAP34657391,"Snap Medical Industries, LLC",Released
6,Feb 8 2023 10:15AM,256910,10,SNAP34667060,"Snap Medical Industries, LLC",Released
7,Feb 8 2023 10:12AM,256909,10,130610,"Virtue Rx, LLC",Released
8,Feb 8 2023 10:11AM,256908,10,Clear-9131841,"ClearSpec, LLC",Released
9,Feb 8 2023 10:11AM,256908,10,Clear-9131843,"ClearSpec, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,256910,Released,2
38,256911,Released,1
39,256912,Released,2
40,256913,Released,1
41,256914,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256910,NaN,NaN,2.0
256911,NaN,NaN,1.0
256912,NaN,NaN,2.0
256913,NaN,NaN,1.0
256914,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,25.0,12.0,15.0
256824,0.0,0.0,1.0
256826,0.0,0.0,113.0
256827,0.0,0.0,16.0
256828,0.0,9.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,25,12,15
256824,0,0,1
256826,0,0,113
256827,0,0,16
256828,0,9,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,25,12,15
1,256824,0,0,1
2,256826,0,0,113
3,256827,0,0,16
4,256828,0,9,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,25,12,15
1,256824,,,1
2,256826,,,113
3,256827,,,16
4,256828,,9,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 8 2023 10:30AM,256914,19,"GUSA Granules USA, Inc."
1,Feb 8 2023 10:26AM,256913,21,"NBTY Global, Inc."
2,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation
4,Feb 8 2023 10:24AM,256911,16,"SHL Pharma, LLC"
5,Feb 8 2023 10:15AM,256910,10,"Snap Medical Industries, LLC"
7,Feb 8 2023 10:12AM,256909,10,"Virtue Rx, LLC"
8,Feb 8 2023 10:11AM,256908,10,"ClearSpec, LLC"
12,Feb 8 2023 10:09AM,256907,10,"Senseonics, Incorporated"
14,Feb 8 2023 10:08AM,256906,10,MedStone Pharma LLC
15,Feb 8 2023 10:04AM,256826,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 8 2023 10:30AM,256914,19,"GUSA Granules USA, Inc.",,,1
1,Feb 8 2023 10:26AM,256913,21,"NBTY Global, Inc.",,,1
2,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation,,,2
3,Feb 8 2023 10:24AM,256911,16,"SHL Pharma, LLC",,,1
4,Feb 8 2023 10:15AM,256910,10,"Snap Medical Industries, LLC",,,2
5,Feb 8 2023 10:12AM,256909,10,"Virtue Rx, LLC",,,1
6,Feb 8 2023 10:11AM,256908,10,"ClearSpec, LLC",,,4
7,Feb 8 2023 10:09AM,256907,10,"Senseonics, Incorporated",,,2
8,Feb 8 2023 10:08AM,256906,10,MedStone Pharma LLC,,,1
9,Feb 8 2023 10:04AM,256826,15,"Person & Covey, Inc.",,,113


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 10:30AM,256914,19,"GUSA Granules USA, Inc.",1,,
1,Feb 8 2023 10:26AM,256913,21,"NBTY Global, Inc.",1,,
2,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation,2,,
3,Feb 8 2023 10:24AM,256911,16,"SHL Pharma, LLC",1,,
4,Feb 8 2023 10:15AM,256910,10,"Snap Medical Industries, LLC",2,,
5,Feb 8 2023 10:12AM,256909,10,"Virtue Rx, LLC",1,,
6,Feb 8 2023 10:11AM,256908,10,"ClearSpec, LLC",4,,
7,Feb 8 2023 10:09AM,256907,10,"Senseonics, Incorporated",2,,
8,Feb 8 2023 10:08AM,256906,10,MedStone Pharma LLC,1,,
9,Feb 8 2023 10:04AM,256826,15,"Person & Covey, Inc.",113,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 10:30AM,256914,19,"GUSA Granules USA, Inc.",1,,
1,Feb 8 2023 10:26AM,256913,21,"NBTY Global, Inc.",1,,
2,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation,2,,
3,Feb 8 2023 10:24AM,256911,16,"SHL Pharma, LLC",1,,
4,Feb 8 2023 10:15AM,256910,10,"Snap Medical Industries, LLC",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 10:30AM,256914,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
1,Feb 8 2023 10:26AM,256913,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation,2.0,NaN,NaN
3,Feb 8 2023 10:24AM,256911,16,"SHL Pharma, LLC",1.0,NaN,NaN
4,Feb 8 2023 10:15AM,256910,10,"Snap Medical Industries, LLC",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 10:30AM,256914,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
1,Feb 8 2023 10:26AM,256913,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation,2.0,0.0,0.0
3,Feb 8 2023 10:24AM,256911,16,"SHL Pharma, LLC",1.0,0.0,0.0
4,Feb 8 2023 10:15AM,256910,10,"Snap Medical Industries, LLC",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4367166,145.0,9.0,0.0
15,1284240,182.0,24.0,25.0
16,770663,3.0,0.0,0.0
18,256892,0.0,1.0,0.0
19,1798047,27.0,21.0,0.0
21,513783,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4367166,145.0,9.0,0.0
1,15,1284240,182.0,24.0,25.0
2,16,770663,3.0,0.0,0.0
3,18,256892,0.0,1.0,0.0
4,19,1798047,27.0,21.0,0.0
5,21,513783,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,145.0,9.0,0.0
1,15,182.0,24.0,25.0
2,16,3.0,0.0,0.0
3,18,0.0,1.0,0.0
4,19,27.0,21.0,0.0
5,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,145.0
1,15,Released,182.0
2,16,Released,3.0
3,18,Released,0.0
4,19,Released,27.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,21
Status,,,,,,
Completed,0.0,25.0,0.0,0.0,0.0,0.0
Executing,9.0,24.0,0.0,1.0,21.0,0.0
Released,145.0,182.0,3.0,0.0,27.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,21
0,Completed,0.0,25.0,0.0,0.0,0.0,0.0
1,Executing,9.0,24.0,0.0,1.0,21.0,0.0
2,Released,145.0,182.0,3.0,0.0,27.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,21
0,Completed,0.0,25.0,0.0,0.0,0.0,0.0
1,Executing,9.0,24.0,0.0,1.0,21.0,0.0
2,Released,145.0,182.0,3.0,0.0,27.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()